## 空调电费查询接口
url为http://www.sdsddk.sdnu.edu.cn/wxapp/pay/queryElectricity

data数据包含三项：
* userXq：用户校区，填写“长清湖校区”“千佛山校区”
* userFj：用户房间号，表示为 宿舍楼号-宿舍楼层-0-房间号，如27号楼503表示为2705002，在楼层号与房间号之间多加一个0
* payType：待查询的类型，1表示照明电费，2表示空调电费

返回的数据对象中，plusElec和feeElec表示当前的电费


## 查询接口

In [60]:
import urllib.request
import urllib.parse
import json

url = "http://www.sdsddk.sdnu.edu.cn/wxapp/pay/queryElectricity"
data=  {
    "userXq":"长清湖校区",
    "userFj":"2003021",
    "payType":"1"
}
#对要发送的数据进行打包
postData = urllib.parse.urlencode(data).encode("utf-8")
#请求体
req = urllib.request.Request(url, postData)
#发起请求
req.add_header( "User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36")
response = urllib.request.urlopen(req)
res = response.read().decode("utf-8")
print(res)
print(json.loads(res)["message"]["feeElec"])

{"success": true,"message": {"feeElec":"0","negElec":"0.00","status":"正常供电","freeElec":"110.10","plusElec":"110.10","room":"2003021"}}
0


In [64]:
## 封装查询接口
def getElec(userFj, payType=1):
    url = "http://www.sdsddk.sdnu.edu.cn/wxapp/pay/queryElectricity"
    data=  {
        "userXq":"长清湖校区",
        "userFj": userFj,
        "payType": payType
    }
    #对要发送的数据进行打包
    postData = urllib.parse.urlencode(data).encode("utf-8")
    #请求体
    req = urllib.request.Request(url, postData)
    #发起请求
    req.add_header( "User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36")
    response = urllib.request.urlopen(req)
    res = response.read().decode("utf-8")
    return json.loads(res)["message"]["plusElec"]

## 读取学生数据

In [1]:
import pandas as pd
import os

In [3]:
filePath = r"A:\desktop\宿舍表.xlsx"

df = pd.read_excel(filePath)

In [70]:
# 研一所有学生的宿舍数据，包括宿舍楼号，房间号，床位号
# 如 27.0	436.0	NaN	谭**	徐**	蔡**
# df

In [13]:
df.drop(axis = 0, index=6, inplace=True)

In [20]:
df.iloc[:,0].values

array([23., 23., 23., 27., 23., 23., 27., 23., 27., 23., 23., 23., 23.,
       27., 23., 23., 27., 23., 27., 23., 27., 23., 27., 26., 27., 27.,
       27., 26., 27., 27., 23., 23., 27., 23., 27., 23., 27., 27., 27.,
       27., 27., 23., 27., 27., 23., 23., 23., 27., 27., 26.])

In [21]:
df.iloc[:,1].values

array([520., 507., 508., 503., 510., 506., 444., 513., 442., 511., 504.,
       503., 514., 504., 505., 516., 436., 515., 524., 512., 501., 509.,
       522., 206., 526., 520., 523., 205., 521., 438., 518., 519., 502.,
       517., 525., 517., 440., 527., 311., 530., 621., 332., 529., 528.,
       330., 334., 523., 310., 532., 204.])

In [37]:
for i in df.iloc[:,0].values:
    '{:0>2d}'.format(int(i))

In [46]:
# 输入楼号与房间号，转化为userFj所需要的输入形式
id_rooms = []
for i, j in zip(df.iloc[:,0].values, df.iloc[:,1].values):
    building_num = '{:0>2d}'.format(int(i))
    s = '{:0>4d}'.format(int(j))
    l = list(s)
    l.insert(2,'0')
    floor_num = ''.join(l)
    id_rooms.append(building_num+floor_num)
    print(building_num+floor_num)


2305020
2305007
2305008
2705003
2305010
2305006
2704044
2305013
2704042
2305011
2305004
2305003
2305014
2705004
2305005
2305016
2704036
2305015
2705024
2305012
2705001
2305009
2705022
2602006
2705026
2705020
2705023
2602005
2705021
2704038
2305018
2305019
2705002
2305017
2705025
2305017
2704040
2705027
2703011
2705030
2706021
2303032
2705029
2705028
2303030
2303034
2305023
2703010
2705032
2602004


In [48]:
id_rooms

['2305020',
 '2305007',
 '2305008',
 '2705003',
 '2305010',
 '2305006',
 '2704044',
 '2305013',
 '2704042',
 '2305011',
 '2305004',
 '2305003',
 '2305014',
 '2705004',
 '2305005',
 '2305016',
 '2704036',
 '2305015',
 '2705024',
 '2305012',
 '2705001',
 '2305009',
 '2705022',
 '2602006',
 '2705026',
 '2705020',
 '2705023',
 '2602005',
 '2705021',
 '2704038',
 '2305018',
 '2305019',
 '2705002',
 '2305017',
 '2705025',
 '2305017',
 '2704040',
 '2705027',
 '2703011',
 '2705030',
 '2706021',
 '2303032',
 '2705029',
 '2705028',
 '2303030',
 '2303034',
 '2305023',
 '2703010',
 '2705032',
 '2602004']

## 批量读取数据

In [66]:
# 获取照明电费
for roomid in id_rooms:
    print(roomid, getElec(roomid))

2305020 327.48
2305007 651.98
2305008 436.07
2705003 114.94
2305010 299.77
2305006 441.43
2704044 156.20
2305013 728.18
2704042 30.44
2305011 914.35
2305004 332.42
2305003 654.44
2305014 417.01
2705004 195.15
2305005 931.67
2305016 511.56
2704036 225.34
2305015 786.09
2705024 113.60
2305012 408.32
2705001 102.44
2305009 673.39
2705022 219.24
2602006 208.02
2705026 255.95
2705020 167.19
2705023 184.65
2602005 210.15
2705021 213.81
2704038 159.93
2305018 354.01
2305019 688.58
2705002 202.55
2305017 540.58
2705025 220.49
2305017 540.58
2704040 35.13
2705027 219.63
2703011 242.22
2705030 219.57
2706021 202.99
2303032 244.67
2705029 153.42
2705028 225.53
2303030 371.56
2303034 332.03
2305023 740.13
2703010 165.82
2705032 159.59
2602004 182.06


In [68]:
# 获取空调电费
for roomid in id_rooms:
    print(roomid, getElec(roomid, payType=2))

2305020 3.42
2305007 .00
2305008 32.29
2705003 62.75
2305010 .00
2305006 30.66
2704044 14.66
2305013 0.00
2704042 0.00
2305011 0.00
2305004 0.00
2305003 0.00
2305014 0.00
2705004 0.00
2305005 0.00
2305016 0.00
2704036 0.00
2305015 0.00
2705024 0.00
2305012 0.00
2705001 0.00
2305009 0.00
2705022 0.00
2602006 0.00
2705026 0.00
2705020 0.00
2705023 0.00
2602005 0.00
2705021 0.00
2704038 0.00
2305018 0.00
2305019 0.00
2705002 0.00
2305017 0.00
2705025 0.00
2305017 0.00
2704040 0.00
2705027 0.00
2703011 0.00
2705030 0.00
2706021 0.00
2303032 0.00
2705029 0.00
2705028 0.00
2303030 0.00
2303034 0.00
2305023 0.00
2703010 0.00
2705032 0.00
2602004 0.00


In [69]:
# 获取空调电费显然太慢了，可能是有解析不出或者json格式不统一的问题
# 上面数据不准确，可能是晚上1点之后不允许爬取，或者其他策略，明天解决吧